In [277]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [278]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [279]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 32

embeddings = CustomEmbedding(vocab_size, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[-0.2546,  0.0346, -0.0045, -0.9555,  0.0711, -0.2120, -1.3931, -1.0510,
          1.9754,  0.9869, -0.7640,  0.5958, -1.9772, -0.0739, -0.7620, -1.1189,
          0.2519,  0.8518,  0.2007, -0.1521, -0.3185, -0.7445, -1.0534,  0.8293,
         -2.2983,  1.1245, -0.3194, -0.9793, -0.0394,  1.7325,  0.3887, -0.6022],
        [-0.4555, -0.7351,  1.1092, -1.1423, -0.4878, -0.4105,  0.0475,  0.3703,
          1.1930, -0.4109,  0.1240,  0.4430, -0.8935,  0.7349,  0.9147, -0.6376,
         -0.3248, -0.1199, -0.2695,  0.8278, -1.5278, -0.2188,  1.4509,  1.3834,
          0.4622, -1.0231, -1.8317, -1.3005, -0.0417, -0.0864, -0.7210, -0.7192]],
       grad_fn=<EmbeddingBackward0>)


In [280]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 32
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

NUM_TRAINING_SEQUENCES = 10000
NUM_VALIDATION_SEQUENCES = 1000

VOCAB = [i for i in range(n_real_tokens)] # non-sos,eos,pad

train_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_TRAINING_SEQUENCES, VOCAB)
train_dataloader = DataLoader(train_rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

val_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_VALIDATION_SEQUENCES, VOCAB)
val_dataloader = DataLoader(val_rand_ds, batch_size = BATCH_SIZE, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [281]:
input, label = next(iter(train_dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 0,  9,  9,  9,  3,  9,  3, 10, 10, 10],
        [ 7,  2, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 2,  6,  2,  0,  6,  6,  4,  8, 10, 10],
        [ 0,  7,  8,  4,  0,  8,  7, 10, 10, 10],
        [ 4,  3,  1,  7,  5,  7,  3,  0,  2,  2],
        [ 3,  7, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 7,  8,  5,  6, 10, 10, 10, 10, 10, 10],
        [ 3,  6,  5,  3, 10, 10, 10, 10, 10, 10],
        [ 3,  6,  2,  0,  0,  4,  6,  4,  4, 10],
        [ 9,  7,  6,  2, 10, 10, 10, 10, 10, 10],
        [ 4,  4,  2,  4,  5,  4, 10, 10, 10, 10],
        [ 6,  4,  9, 10, 10, 10, 10, 10, 10, 10],
        [ 7,  0,  6,  2,  9, 10, 10, 10, 10, 10],
        [ 9,  5,  4,  0,  5,  3,  1,  8,  2,  5],
        [ 1,  5,  4,  1,  8,  6,  8,  2,  5, 10],
        [ 6,  8,  1,  1, 10, 10, 10, 10, 10, 10],
        [ 0,  4,  2, 10, 10, 10, 10, 10, 10, 10],
        [ 5,  8,  3,  5,  8,  8, 10, 10, 10, 10],
        [ 4,  9,  1, 10, 10, 10, 10, 10, 10, 10],
        [ 4,  3,  2,  2,  8, 10, 10, 10, 10, 10],


In [282]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    max_context_window = MAX_CONTEXT_WINDOW)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [283]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.train()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    for (source, target), label in tqdm(dataloader):

        # FORWARD
        pred_logits = model(source, target)

        # pred_logits.shape: [batch_size, seq_len, vocab_size]
        # label.shape: [batch_size, seq_len]

        # CrossEntropyLoss (loss_fn) only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

        # LOG
        with torch.no_grad():
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_gold_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.eval()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    with torch.no_grad():
        
        for (source, target), label in tqdm(dataloader):
            
            # FORWARD
            pred_logits = model(source, target)
            batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

            # LOG
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_autoregressive_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    """
    """
    model.eval()

    with torch.no_grad():
        ...

In [284]:
EPOCHS = 10

training_losses = list()
training_sequence_accuracies = list()
training_token_accuracies = list()

gold_validation_losses = list()
gold_validation_sequence_accuracies = list()
gold_validation_token_accuracies = list()

for i in range(EPOCHS):
    # print(f'Running epoch {i+1}...')

    training_loss, training_sequence_accuracy, training_token_accuracy = run_train_epoch(train_dataloader, model, loss_fn, optim, calculate_sequence_accuracy = True, calculate_token_accuracy = True)

    training_losses.append(training_loss)
    training_sequence_accuracies.append(training_sequence_accuracy)
    training_token_accuracies.append(training_token_accuracy)

    gold_val_loss, gold_val_sequence_accuracy, gold_val_token_accuracy = run_gold_validation_loop(val_dataloader, model, loss_fn, calculate_sequence_accuracy = True, calculate_token_accuracy = True)
    gold_validation_losses.append(gold_val_loss)
    gold_validation_sequence_accuracies.append(gold_val_sequence_accuracy)
    gold_validation_token_accuracies.append(gold_val_token_accuracy)

print(training_losses)
print(training_sequence_accuracies)
print(training_token_accuracies)

print()

print(gold_validation_losses)
print(gold_validation_sequence_accuracies)
print(gold_validation_token_accuracies)

100%|██████████| 32/32 [00:00<00:00, 320.11it/s]

[3.75580693321228, 2.3439517858505248, 2.2022707111358644, 1.9040485794067383, 1.7696340825080872, 1.5165319305419922, 1.4700859122276306, 1.5675110280036926, 1.3338396703243256, 1.3710929999351502]
[0.0047, 0.0106, 0.0154, 0.0229, 0.0246, 0.0336, 0.0358, 0.0339, 0.039, 0.0402]
[0.5088518140754772, 0.5593336002329306, 0.5653421512898994, 0.5767625400991543, 0.5823602393811123, 0.5882695386406637, 0.5925979892175434, 0.5878772415804053, 0.5971230881281865, 0.5970397138268361]

[4.347921878814697, 2.7143004779815674, 4.296113105773926, 1.32000705909729, 1.3183626251220704, 2.2834864110946653, 4.3253308944702145, 1.0522940735816955, 2.3112687339782716, 1.7282714982032776]
[0.002, 0.006, 0.001, 0.024, 0.053, 0.025, 0.001, 0.047, 0.013, 0.026]
[0.4877468910021946, 0.540416971470373, 0.4786942209217264, 0.5897951719092904, 0.5922640819312363, 0.5695866861741039, 0.496525237746891, 0.6012253108997806, 0.5633686905632772, 0.5883321141185077]
